In [4]:
import pandas as pd
import sys
sys.path.append('..')
import data_loader as dl
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [59]:
def enumerate_y_peaks(row):
    enumeration = []
    Y_and_B = row["Matched Ion Series"].split(";")
    
    if len(Y_and_B) == 2: #if there are y and b peaks
        if "y" in Y_and_B[1]:
            peaks = Y_and_B[1]
        elif "y" in Y_and_B[0]:
            peaks = Y_and_B[0]
            
        peak_list = peaks.strip('][').split(",")

        for peak in peak_list:
            enumer_string = peak.split("+")[0]
            enumer = int(re.search(r'\d+', enumer_string).group(0))
            enumeration.append(enumer)
                
    else: #if there are either y or b peaks
        if "y" in Y_and_B[0]: #if it's a y peak
            peaks = Y_and_B[0]
            peak_list = peaks.strip('][').split(",")

            for peak in peak_list:
                enumer_string = peak.split("+")[0]
                enumer = int(re.search(r'\d+', enumer_string).group(0))
                enumeration.append(enumer)
        
        else: #if it's a b peak
            pass

        
    return enumeration  

In [62]:
def enumerate_b_peaks(row):
    enumeration = []
    Y_and_B = row["Matched Ion Series"].split(";")
    
    if len(Y_and_B) == 2: #if there are y and b peaks
        if "b" in Y_and_B[1]:
            peaks = Y_and_B[1]
        elif "b" in Y_and_B[0]:
            peaks = Y_and_B[0]
            
        peak_list = peaks.strip('][').split(",")

        for peak in peak_list:
            enumer_string = peak.split("+")[0]
            enumer = int(re.search(r'\d+', enumer_string).group(0))
            enumeration.append(enumer)
                
    else: #if there are either y or b peaks
        if "b" in Y_and_B[0]: #if it's a b peak
            peaks = Y_and_B[0]
            peak_list = peaks.strip('][').split(",")

            for peak in peak_list:
                enumer_string = peak.split("+")[0]
                enumer = int(re.search(r'\d+', enumer_string).group(0))
                enumeration.append(enumer)
        
        else: #if it's a y peak
            pass

        
    return enumeration    

In [7]:
def count_consecutive(row, col_name):
    peak_list = row[col_name]
    retlist = []
    count = 1
    # Avoid IndexError for  random_list[i+1]
    for i in range(len(peak_list) - 1):
        # Check if the next number is consecutive
        if peak_list[i] + 1 == peak_list[i+1]:
            count += 1
        else:
            # If it is not append the count and restart counting
            retlist.append(count)
            count = 1
    # Since we stopped the loop one early append the last count
    retlist.append(count)
    return retlist

In [8]:
def get_max_consec(row, col_name):
    return max(row[col_name])

In [27]:
def perc_ladder_peaks(row):
    peak_link_list = row['Ypeak_enum']
    consec_peak_list = row['consecutive_y_peaks']
    consec_peak_list = list(filter(lambda a: a != 1, consec_peak_list))
    if len(peak_link_list) == 0:
        perc = -1 #this is for when there no Y peaks
    else:
        perc = sum(consec_peak_list)/len(peak_link_list)
    return perc

In [56]:
def calc_consecutive_peaks(df):
    sc = df[['scan','Matched Ion Series']]
#     sc.assign(Bpeak_enum=enumerate_b_peaks, axis=1)
    sc["Bpeak_enum"] = sc.apply(enumerate_b_peaks, axis=1)
    sc['consecutive_b_peaks'] = sc.apply(lambda row: count_consecutive(row, 'Bpeak_enum'), axis=1)

    sc["Ypeak_enum"] = sc.apply(enumerate_y_peaks, axis=1)
    sc['consecutive_y_peaks'] = sc.apply(lambda row: count_consecutive(row, 'Ypeak_enum'), axis=1)

    sc['max_consecutive_b'] = sc.apply(lambda row: get_max_consec(row, "consecutive_b_peaks"), axis=1)
    sc['max_consecutive_y'] = sc.apply(lambda row: get_max_consec(row, "consecutive_y_peaks"), axis=1)

    sc['perc_in_ladder'] = sc.apply(perc_ladder_peaks, axis=1)
    sc = sc[['scan','max_consecutive_b','max_consecutive_y','perc_in_ladder']]

    

    return sc

In [63]:
sc2ng_rep1 = dl.clean_metamorph('2ng_rep1')

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [64]:
calc_consecutive_peaks(sc2ng_rep1)

<ipython-input-56-9b3e882902f0>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sc["Bpeak_enum"] = sc.apply(enumerate_b_peaks, axis=1)
<ipython-input-56-9b3e882902f0>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sc['consecutive_b_peaks'] = sc.apply(lambda row: count_consecutive(row, 'Bpeak_enum'), axis=1)
<ipython-input-56-9b3e882902f0>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

,scan,max_consecutive_b,max_consecutive_y,perc_in_ladder
0,16668,8,20,1.000000
1,27140,8,13,1.000000
2,32708,9,11,1.000000
3,33439,10,15,1.000000
4,13306,3,22,1.000000
...,...,...,...,...
21696,9453,2,2,0.666667
21697,9033,2,1,0.000000
21698,7282,1,1,0.000000
21699,15418,1,3,0.750000
